In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ThermoFit as therm
import OneDFit as od
import numpy as np

tc, pc, rhoc = 215., 338., 0.05939
mys = od.SpinodalFit("./Data/compiledEOSAll",230.)
mys.setCriticalPt(tc,pc,rhoc)
mys.fitParams["A(T) params"] = np.poly1d([-0.0822462,-0.0547873])
mys.fitParams["P(T) params"] = np.poly1d([-7.61e-05*tc*tc,0.0305542*tc,-292.0177]) #unit conversion
Model = therm.ThermoPPty("./Data/testdensity.dat",Tc=tc,Pc=pc,Rhoc=rhoc,spinodal=mys)

In [3]:
x = np.loadtxt('./Data/test_fit')
Model.setValues(x)
print(Model.__str__(T=200.,P=-100.))

<At this given state with T=200.000000 and P=-100.000000, the response functions are the following>		
Density is 961.730587 kg/m^3
c_P is 4104.330509 J.(kgK)^-1
c_V is 7004.961557 J.(kgK)^-1
k_T is -0.000206 MPa^-1
alpha_p		is 0.001695 K^-1
c_s is nan

